# Steady-State Subduction Zone Setup

Authors: Kidus Teshome, Cian Wilson

## Themes and variations

Some variations from the previous notebook

* Try higher resolution versions of the benchmark cases
* Varying the coupling depth
* Try some cases from the global suite using the data in `allsz_params`

### Preamble

Let's start by adding the path to the modules in the `python` folder to the system path (so we can find the our custom modules).

In [ ]:
import sys, os
basedir = ''
if "__file__" in globals(): basedir = os.path.dirname(__file__)
sys.path.append(os.path.join(basedir, os.path.pardir, os.path.pardir, 'python'))

Then load everything we need from `sz_problem` and also set our default plotting preferences.

In [ ]:
import utils
from sz_problems.sz_params import default_params, allsz_params
from sz_problems.sz_slab import create_slab
from sz_problems.sz_geometry import create_sz_geometry
from sz_problems.sz_steady_isoviscous import SteadyIsoSubductionProblem
from sz_problems.sz_steady_dislcreep import SteadyDislSubductionProblem
import pathlib
output_folder = pathlib.Path(os.path.join(basedir, "output"))
output_folder.mkdir(exist_ok=True, parents=True)

### Higher Resolution

#### Benchmark case 1

In [ ]:
xs = [0.0, 140.0, 240.0, 400.0]
ys = [0.0, -70.0, -120.0, -200.0]
lc_depth = 40
uc_depth = 15
coast_distance = 0
extra_width = 0
sztype = 'continental'
io_depth_1 = 139
A      = 100.0      # age of subducting slab (Myr)
qs     = 0.065      # surface heat flux (W/m^2)
Vs     = 100.0      # slab speed (mm/yr)

In [ ]:
resscale2 = 2.5
slab_resscale2 = create_slab(xs, ys, resscale2, lc_depth)
geom_resscale2 = create_sz_geometry(slab_resscale2, resscale2, sztype, io_depth_1, extra_width, 
                           coast_distance, lc_depth, uc_depth)
sz_case1_resscale2 = SteadyIsoSubductionProblem(geom_resscale2, A=A, Vs=Vs, sztype=sztype, qs=qs)

In [ ]:
print("\nSolving steady state flow with isoviscous rheology...")
sz_case1_resscale2.solve()

In [ ]:
diag_resscale2 = sz_case1_resscale2.get_diagnostics()

print('')
print('{:<12} {:<12} {:<12} {:<12} {:<12} {:<12}'.format('resscale', 'T_ndof', 'T_{200,-100}', 'Tbar_s', 'Tbar_w', 'Vrmsw'))
print('{:<12.4g} {:<12d} {:<12.4f} {:<12.4f} {:<12.4f} {:<12.4f}'.format(resscale2, *diag_resscale2.values()))

For comparison here are the values reported for case 1 using [TerraFERMA](https://terraferma.github.io) in [Wilson & van Keken, 2023](http://dx.doi.org/10.1186/s40645-023-00588-6):

| `resscale` | $T_{\text{ndof}} $ | $T_{(200,-100)}^*$ | $\overline{T}_s^*$ | $ \overline{T}_w^* $ |  $V_{\text{rms},w}^*$ |
| - | - | - | - | - | - |
| 2.0 | 21403  | 517.17 | 451.83 | 926.62 | 34.64 |
| 1.0 | 83935  | 516.95 | 451.71 | 926.33 | 34.64 |
| 0.5 | 332307 | 516.86 | 451.63 | 926.15 | 34.64 |

In [ ]:
plotter_case1_resscale2 = utils.plot.plot_scalar(sz_case1_resscale2.T_i, scale=sz_case1_resscale2.T0, gather=True, cmap='coolwarm', scalar_bar_args={'title': 'Temperature (deg C)', 'bold':True})
utils.plot.plot_vector_glyphs(sz_case1_resscale2.vw_i, plotter=plotter_case1_resscale2, gather=True, factor=0.05, color='k', scale=utils.mps_to_mmpyr(sz_case1_resscale2.v0))
utils.plot.plot_vector_glyphs(sz_case1_resscale2.vs_i, plotter=plotter_case1_resscale2, gather=True, factor=0.05, color='k', scale=utils.mps_to_mmpyr(sz_case1_resscale2.v0))
utils.plot.plot_geometry(sz_case1_resscale2.geom, plotter=plotter_case1_resscale2, color='green', width=2)
utils.plot.plot_couplingdepth(sz_case1_resscale2.geom.slab_spline, plotter=plotter_case1_resscale2, render_points_as_spheres=True, point_size=10.0, color='green')
utils.plot.plot_show(plotter_case1_resscale2)
utils.plot.plot_save(plotter_case1_resscale2, output_folder / "sz_steady_tests_case1_resscale2_solution.png")

In [ ]:
# clean up
del plotter_case1_resscale2
del sz_case1_resscale2
del geom_resscale2

#### Benchmark case 2

In [ ]:
io_depth_2 = 154.0
geom_case2_resscale2 = create_sz_geometry(slab_resscale2, resscale2, sztype, io_depth_2, extra_width, 
                                          coast_distance, lc_depth, uc_depth)
sz_case2_resscale2 = SteadyDislSubductionProblem(geom_case2_resscale2, A=A, Vs=Vs, sztype=sztype, qs=qs)

In [ ]:
print("\nSolving steady state flow with dislocation creep rheology...")
sz_case2_resscale2.solve()

In [ ]:
diag_case2_resscale2 = sz_case2_resscale2.get_diagnostics()

print('')
print('{:<12} {:<12} {:<12} {:<12} {:<12} {:<12}'.format('resscale', 'T_ndof', 'T_{200,-100}', 'Tbar_s', 'Tbar_w', 'Vrmsw'))
print('{:<12.4g} {:<12d} {:<12.4f} {:<12.4f} {:<12.4f} {:<12.4f}'.format(resscale2, *diag_case2_resscale2.values()))

For comparison here are the values reported for case 2 using [TerraFERMA](https://terraferma.github.io) in [Wilson & van Keken, 2023](http://dx.doi.org/10.1186/s40645-023-00588-6):

| `resscale` | $T_{\text{ndof}} $ | $T_{(200,-100)}^*$ | $\overline{T}_s^*$ | $ \overline{T}_w^* $ |  $V_{\text{rms},w}^*$ |
| - | - | - | - | - | - |
| 2.0 | 21403  | 683.05 | 571.58 | 936.65 | 40.89 |
| 1.0 | 83935 | 682.87 | 572.23 | 936.11 | 40.78 |
| 0.5 | 332307 | 682.80 | 572.05 | 937.37 | 40.77 |

In [ ]:
plotter_case2_resscale2 = utils.plot.plot_scalar(sz_case2_resscale2.T_i, scale=sz_case2_resscale2.T0, gather=True, cmap='coolwarm', scalar_bar_args={'title': 'Temperature (deg C)', 'bold':True})
utils.plot.plot_vector_glyphs(sz_case2_resscale2.vw_i, plotter=plotter_case2_resscale2, gather=True, factor=0.05, color='k', scale=utils.mps_to_mmpyr(sz_case2_resscale2.v0))
utils.plot.plot_vector_glyphs(sz_case2_resscale2.vs_i, plotter=plotter_case2_resscale2, gather=True, factor=0.05, color='k', scale=utils.mps_to_mmpyr(sz_case2_resscale2.v0))
utils.plot.plot_geometry(sz_case2_resscale2.geom, plotter=plotter_case2_resscale2, color='green', width=2)
utils.plot.plot_couplingdepth(sz_case2_resscale2.geom.slab_spline, plotter=plotter_case2_resscale2, render_points_as_spheres=True, point_size=10.0, color='green')
utils.plot.plot_show(plotter_case2_resscale2)
utils.plot.plot_save(plotter_case2_resscale2, output_folder / "sz_steady_tests_case2_resscale2_solution.png")

In [ ]:
# clean up
del plotter_case2_resscale2
del sz_case2_resscale2
del geom_case2_resscale2
del slab_resscale2

### Vary coupling depth

We will now re-use all of the parameters for case 2 but vary the coupling depth by passing in an additional keyword argument `coupling_depth` to `create_slab`.  The rest of the solution procedure is the same as before.

Let's loop over a series of coupling depths to see how varying it changes the solution.

In [ ]:
# set a list of coupling depths to try
coupling_depths = [60.0, 80.0, 100.0]
resscale3 = 3.0

# set up a list to save the diagnostics from each
diagnostics = []
# loop over the couplings depths
for coupling_depth in coupling_depths:
    # create the slab object, all of the input arguments are the same as in case 2
    # but this time we also pass in the coupling_depth keyword argument to override
    # the default value (80 km)
    slab_dc = create_slab(xs, ys, resscale3, lc_depth, coupling_depth=coupling_depth)
    # set up the geometry
    geom_dc = create_sz_geometry(slab_dc, resscale3, sztype, io_depth_2, extra_width, 
                                            coast_distance, lc_depth, uc_depth)
    # set up the subduction zone problem
    sz_dc = SteadyDislSubductionProblem(geom_dc, A=A, Vs=Vs, sztype=sztype, qs=qs)

    # solve the steady state problem
    if sz_dc.comm.rank == 0: print(f"\nSolving steady state flow with coupling depth = {coupling_depth}km...")
    sz_dc.solve()

    # retrieve the diagnostics
    diagnostics.append(sz_dc.get_diagnostics())

    # plot the solution
    plotter_dc = utils.plot.plot_scalar(sz_dc.T_i, scale=sz_dc.T0, gather=True, cmap='coolwarm', 
                                   scalar_bar_args={'title': 'Temperature (deg C)', 'bold':True})
    utils.plot.plot_vector_glyphs(sz_dc.vw_i, plotter=plotter_dc, gather=True, factor=0.05, color='k', 
                             scale=utils.mps_to_mmpyr(sz_dc.v0))
    utils.plot.plot_vector_glyphs(sz_dc.vs_i, plotter=plotter_dc, gather=True, factor=0.05, color='k', 
                             scale=utils.mps_to_mmpyr(sz_dc.v0))
    utils.plot.plot_geometry(sz_dc.geom, plotter=plotter_dc, color='green', width=2)
    utils.plot.plot_couplingdepth(sz_dc.geom.slab_spline, plotter=plotter_dc, render_points_as_spheres=True, 
                             point_size=10.0, color='green')
    utils.plot.plot_show(plotter_dc)
    utils.plot.plot_save(plotter_dc, output_folder / f"sz_steady_tests_dc{coupling_depth}_solution.png")

    # clean up
    del plotter_dc
    del sz_dc
    del geom_dc
    del slab_dc


As well as visualizing the solutions we can see what effect varying the coupling depth has on the global diagnostics from the benchmark.

In [ ]:
# print the varying coupling depth output
print('')
print('{:<12} {:<12} {:<12} {:<12} {:<12} {:<12}'.format('d_c', 'T_ndof', 'T_{200,-100}', 'Tbar_s', 'Tbar_w', 'Vrmsw'))
for dc, diag in zip(coupling_depths, diagnostics):
    print('{:<12.4g} {:<12d} {:<12.4f} {:<12.4f} {:<12.4f} {:<12.4f}'.format(dc, *diag.values()))

Note the dramatic drop in temperature at (200, -100), `T_{200,-100}`, once the coupling depth reaches 100km.

### Global suite examples

#### Alaska Peninsula (dislocation creep, low res)

In [ ]:
resscale_ak = 5.0
szdict_ak = allsz_params['01_Alaska_Peninsula']
slab_ak = create_slab(szdict_ak['xs'], szdict_ak['ys'], resscale_ak, szdict_ak['lc_depth'])
geom_ak = create_sz_geometry(slab_ak, resscale_ak, szdict_ak['sztype'], szdict_ak['io_depth'], szdict_ak['extra_width'], 
                             szdict_ak['coast_distance'], szdict_ak['lc_depth'], szdict_ak['uc_depth'])
sz_ak = SteadyDislSubductionProblem(geom_ak, **szdict_ak)

In [ ]:
print("\nSolving steady state flow with isoviscous rheology...")
sz_ak.solve()

In [ ]:
plotter_ak = utils.plot.plot_scalar(sz_ak.T_i, scale=sz_ak.T0, gather=True, cmap='coolwarm', scalar_bar_args={'title': 'Temperature (deg C)', 'bold':True})
utils.plot.plot_vector_glyphs(sz_ak.vw_i, plotter=plotter_ak, gather=True, factor=0.1, color='k', scale=utils.mps_to_mmpyr(sz_ak.v0))
utils.plot.plot_vector_glyphs(sz_ak.vs_i, plotter=plotter_ak, gather=True, factor=0.1, color='k', scale=utils.mps_to_mmpyr(sz_ak.v0))
utils.plot.plot_geometry(sz_ak.geom, plotter=plotter_ak, color='green', width=2)
utils.plot.plot_couplingdepth(sz_ak.geom.slab_spline, plotter=plotter_ak, render_points_as_spheres=True, point_size=10.0, color='green')
utils.plot.plot_show(plotter_ak)
utils.plot.plot_save(plotter_ak, output_folder / "sz_steady_tests_ak_solution.png")

In [ ]:
eta_ak = sz_ak.project_dislocationcreep_viscosity()
plotter_eta_ak = utils.plot.plot_scalar(eta_ak, scale=sz_ak.eta0, gather=True, log_scale=True, show_edges=True, scalar_bar_args={'title': 'Viscosity (Pa s)', 'bold':True})
utils.plot.plot_geometry(sz_ak.geom, plotter=plotter_eta_ak, color='green', width=2)
utils.plot.plot_couplingdepth(sz_ak.geom.slab_spline, plotter=plotter_eta_ak, render_points_as_spheres=True, point_size=10.0, color='green')
utils.plot.plot_show(plotter_eta_ak)
utils.plot.plot_save(plotter_eta_ak, output_folder / "sz_steady_tests_ak_eta.png")

In [ ]:
# clean up
del plotter_eta_ak
del sz_ak
del geom_ak
del slab_ak

#### N Antilles (dislocation creep, low res)

In [ ]:
resscale_ant = 5.0
szdict_ant = allsz_params['19_N_Antilles']
slab_ant = create_slab(szdict_ant['xs'], szdict_ant['ys'], resscale_ant, szdict_ant['lc_depth'])
geom_ant = create_sz_geometry(slab_ant, resscale_ant, szdict_ant['sztype'], szdict_ant['io_depth'], szdict_ant['extra_width'], 
                              szdict_ant['coast_distance'], szdict_ant['lc_depth'], szdict_ant['uc_depth'])
sz_ant = SteadyDislSubductionProblem(geom_ant, **szdict_ant)

In [ ]:
print("\nSolving steady state flow with isoviscous rheology...")
sz_ant.solve()

In [ ]:
plotter_ant = utils.plot.plot_scalar(sz_ant.T_i, scale=sz_ant.T0, gather=True, cmap='coolwarm', scalar_bar_args={'title': 'Temperature (deg C)', 'bold':True})
utils.plot.plot_vector_glyphs(sz_ant.vw_i, plotter=plotter_ant, gather=True, factor=0.25, color='k', scale=utils.mps_to_mmpyr(sz_ant.v0))
utils.plot.plot_vector_glyphs(sz_ant.vs_i, plotter=plotter_ant, gather=True, factor=0.25, color='k', scale=utils.mps_to_mmpyr(sz_ant.v0))
utils.plot.plot_geometry(sz_ant.geom, plotter=plotter_ant, color='green', width=2)
utils.plot.plot_couplingdepth(sz_ant.geom.slab_spline, plotter=plotter_ant, render_points_as_spheres=True, point_size=10.0, color='green')
utils.plot.plot_show(plotter_ant)
utils.plot.plot_save(plotter_ant, output_folder / "sz_steady_tests_ant_solution.png")

In [ ]:
eta_ant = sz_ant.project_dislocationcreep_viscosity()
plotter_eta_ant = utils.plot.plot_scalar(eta_ant, scale=sz_ant.eta0, gather=True, log_scale=True, show_edges=True, scalar_bar_args={'title': 'Viscosity (Pa s)', 'bold':True})
utils.plot.plot_geometry(sz_ant.geom, plotter=plotter_eta_ant, color='green', width=2)
utils.plot.plot_couplingdepth(sz_ant.geom.slab_spline, plotter=plotter_eta_ant, render_points_as_spheres=True, point_size=10.0, color='green')
utils.plot.plot_show(plotter_eta_ant)
utils.plot.plot_save(plotter_eta_ant, output_folder / "sz_steady_tests_ant_eta.png")

In [ ]:
# clean up
del plotter_eta_ant
del sz_ant
del geom_ant
del slab_ant